
# Session 2 — Manipulation des données sous Python
**Objectifs**
- Charger des données tabulaires (CSV) de manière robuste
- Inspecter: `shape`, `head`, `info`, `describe`, `dtypes`
- Nettoyer: NA, types, valeurs aberrantes
- Transformer: colonnes dérivées, groupby/agg, pivot, merge/join
- Exporter: CSV/Parquet

> Dataset: utilisez votre `FB.csv` (ou un CSV similaire). Les cellules incluent des garde-fous et des tests.


## 0. Imports & options

In [1]:

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 8)
pd.set_option("display.width", 120)
print(pd.__version__)


1.5.3


## 1. Chargement robuste d'un CSV

In [ ]:

from pathlib import Path

path = Path("FB.csv")  # changez si besoin
if not path.exists():
    # crée un petit CSV de démo si FB.csv est absent
    demo = pd.DataFrame({
        "comment":[5,2,0,1],
        "like":[10, np.nan, 3, 2],
        "share":[1, 1, np.nan, 0],
        "Category":[1,2,1,3],
        "Post Month":[9,10,10,11],
        "Post Hour":[15,20,10,17],
        "Type":["Photo","Status","Photo","Video"]
    })
    demo.to_csv(path, index=False)

# Auto-détection du séparateur
df = pd.read_csv(path, sep=None, engine="python")
df.head()


## 2. Inspecter & diagnostiquer

In [ ]:

print(df.shape)
print(df.dtypes)
print(df.isna().sum())
df.describe(include="all")


## 3. Nettoyage minimal (types & NA)

In [ ]:

# Coercition en numérique si colonnes censées être numériques
numeric_candidates = ["comment","like","share","Category","Post Month","Post Hour"]
for c in numeric_candidates:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Imputation simple des numériques par la moyenne
num_cols = df.select_dtypes(include=["number"]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].mean(numeric_only=True))

# Colonne dérivée vectorisée
if {"comment","share"}.issubset(df.columns):
    df["action"] = df["comment"] + df["share"]

# Vérifications
assert df.select_dtypes(include=["number"]).isna().sum().sum() == 0
assert "action" not in df.columns or (df["action"] >= 0).all()
print("✓ Nettoyage minimal OK")
df.head()


## 4. Filtrer, trier, sélectionner

In [ ]:

# Sélection colonnes
cols = [c for c in ["comment","like","share","action","Type"] if c in df.columns]
view = df[cols].copy()

# Filtre
high_like = view[view["like"] > view["like"].median()] if "like" in view else view

# Tri
ordered = high_like.sort_values(by=cols[0]) if cols else df
ordered.head()


## 5. Groupby / agrégations

In [ ]:

if "Type" in df.columns:
    agg = (df.groupby("Type")
             .agg(n=("Type","count"),
                  like_mean=("like","mean"),
                  action_sum=("action","sum") if "action" in df else ("comment","sum"))
             .reset_index())
    display(agg)
else:
    print("Pas de colonne 'Type' — passez à l'étape suivante.")


## 6. Jointures (merge) — mini démo

In [ ]:

# Jeu jouet pour merge
users = pd.DataFrame({"user_id":[1,2,3], "country":["FR","FR","DE"]})
events = pd.DataFrame({"user_id":[1,1,3,4], "clicks":[5,2,1,7]})
joined = users.merge(events, on="user_id", how="left")
joined



### 💪 À vous 1 — Pipeline end-to-end
1. Recharger `FB.csv` (ou un autre CSV) dans `df2`.
2. Forcer en numérique `["comment","like","share"]` et imputer.
3. Créer `engagement = like + share + comment`.
4. Calculer, par `Type`, le `engagement` moyen et le nombre de lignes.


In [ ]:

# TODO
df2 = pd.read_csv(path, sep=None, engine="python")

for c in ["comment","like","share"]:
    if c in df2.columns:
        df2[c] = pd.to_numeric(df2[c], errors="coerce")

num_cols2 = df2.select_dtypes(include=["number"]).columns
df2[num_cols2] = df2[num_cols2].fillna(df2[num_cols2].mean(numeric_only=True))

if {"comment","like","share"}.issubset(df2.columns):
    df2["engagement"] = df2["comment"] + df2["like"] + df2["share"]

if "Type" in df2.columns:
    sol = df2.groupby("Type").agg(n=("Type","count"),
                                  engagement_mean=("engagement","mean")).reset_index()
    display(sol.head())
else:
    sol = None

# tests
if sol is not None:
    assert {"Type","n","engagement_mean"}.issubset(sol.columns)
print("✓ À vous 1 — OK")


## 7. Export (CSV / Parquet)

In [ ]:

df.to_csv("FB_clean.csv", index=False)
try:
    df.to_parquet("FB_clean.parquet", index=False)
    print("✓ Exporté en CSV et Parquet")
except Exception as e:
    print("Parquet indisponible (pyarrow non installé?):", e)



## 8. Bonus — Validation rapide & asserts utiles
Ajoutez quelques vérifications qui échouent tôt si les hypothèses sont fausses.


In [ ]:

# Exemples
if "engagement" in df2.columns:
    assert (df2["engagement"] >= 0).all(), "Engagement négatif impossible"
if "Post Hour" in df2.columns:
    ok = df2["Post Hour"].between(0, 23).all()
    assert ok, "Post Hour doit être entre 0 et 23"
print("✓ Asserts de validation passés")
